In [3]:
import os
import pandas as pd
import datetime
import urllib
import requests
pd.set_option('display.max_columns', 500)

In [39]:
def read_last_30days_data_from_url():
    start_date=datetime.datetime.now() - datetime.timedelta(days=30)
    date_string=start_date.strftime("%Y-%m-%d")
    url = 'https://data.boston.gov/api/3/action/datastore_search_sql?sql=SELECT * from "2968e2c0-d479-49ba-a884-4ef523ada3c0" WHERE open_dt >=\''+date_string+'\' ORDER BY open_dt desc' 
    fileobj = requests.get(url)
    json_response = fileobj.json()
    current_data=pd.DataFrame(json_response['result']['records'])
    current_data=current_data.sort_values(by='_id')
    current_data=current_data.drop('_full_text',axis=1)
    return current_data

def find_prev_file():
    try:
        with open('C:/Users/rwagh/Desktop/City of Boston data asssesment/Output/cases_311_data.csv') as fh:
            return pd.read_csv(fh)
    except FileNotFoundError:
        print('file not found')
        return None
    
def allign_data_type_of_current_data(current_data,prev_data):
    for column in current_data.columns:
        current_data[column] = current_data[column].astype(prev_data[column].dtype)
    return current_data

def add_delta_to_prev_data(current_data,prev_data):
    current_data=allign_data_type_of_current_data(current_data,prev_data)
    merged=pd.merge(current_data, prev_data,on='_id',how='left',left_index=False,right_index=False, indicator=True)
    to_be_appended = merged.loc[merged._merge == 'left_only']
    to_be_appended = to_be_appended[to_be_appended.columns[~to_be_appended.columns.str.endswith('_y')]]
    to_be_appended.columns = to_be_appended.columns.str.rstrip('_x')
    to_be_appended = to_be_appended.drop(['_merge'], axis=1) 
    return prev_data.append(to_be_appended)

current_data=read_last_30days_data_from_url()
prev_data=find_prev_file()
if prev_data is None:
     current_data.to_csv('Output/cases_311_data.csv',index=False)
else:
    new_data=add_delta_to_prev_data(current_data,prev_data)
    #new_data = new_data.drop_duplicates()
    new_data=new_data.sort_values(by='_id')
    new_data.to_csv('Output/cases_311_data.csv',index=False)